# A lőcsei polgárság kapcsolattörténeti vizsgálata (1614-1670)

Első lépésként a pandas könyvtár segítségével beolvasom az adatgyűjtés során készített Excel fájlt. 

In [10]:
import pandas
import networkx as nx
from pyvis import network
import matplotlib.pyplot as plt
import math
import textwrap
import warnings
warnings.filterwarnings('ignore')

records = pandas.read_excel('sources/zipserbuch.xlsx', sheet_name='lőcsei vonatkozású alkalmi')
records.columns = records.columns.str.strip()
records.set_index('RMNy',  inplace=True)
records.head()

,ID,év,típusa,esemény,a nyomtatvány rövid leírása,a nyomtatvány címzettjei,szerzők és a nyomtatványban említett kapcsolódó nevek
RMNy,,,,,,,
1076,4,1614,Gyászversek,halál,Thurzó Kristóf főispán felett mondott gyászversek,Thurzó Kristóf,"Holtzmann, Stephanus"
1230,18,1620,Oktató versek,beiktatás,A versek a lőcsei városi magistrátus és az ala...,"Cramer, Caspar id.","Musculus, Wolfgang"
1231,19,1620,Gyászbeszéd,halál,Stephanus Holtzmann (Xylander) szepesváraljai ...,"Holtzmann, Stephanus","Zabeler, Peter; Horlerus, Stanislaus"
1254,20,1621,Lakodalmi ének,esküvő,Daniel Cramer és Christina Wolf 1621. szeptemb...,"Cramer, Daniel; Wolf, Christina","Musculus, Wolfgang; Wolf, Johann"
1255,21,1621,Oktató versek,általános üdvözlés,Tizenkét vers az elöljáróknak szükséges tizenk...,NaN,"Musculus, Wolfgang"


In [11]:
citizens = pandas.read_excel('sources/zipserbuch.xlsx', sheet_name='személyek')

# A többszörös szóközök és a szögletes zárójelek eltüntetése
for index, item in citizens.iterrows():
    name = " ".join(item['név'].split())
    name = name.replace('[', '')
    name = name.replace(']', '')
   
    citizens.at[index, 'név'] = name
        
citizens.head()

,id,név,születési hely,névváltozat,foglalkozás,életrajzi hivatkozás,rövid leírás,RMNy (alkalmi)
0,1001,szervezet a besztercebányai evangélikus iskola...,NaN,NaN,vizsga,NaN,A tételeket 1642 májusában vitatták meg a besz...,1934
1,1002,szervezet a lőcsei evangélikus gimnázium előad...,NaN,NaN,program,NaN,Meghívó a lőcsei evangélikus gimnázium hallgat...,2223
2,1003,szervezet a lőcsei evangélikus gimnázium ünnep...,NaN,NaN,program,NaN,A lőcsei evangélikus gimnázium 1666. január 30...,3295
3,1004,szervezet a lőcsei evangélikus gimnázium vizsg...,NaN,NaN,vizsga,NaN,A nyomtatvány a lőcsei evangélikus gimnázium 1...,3296
4,1005,szervezet a lőcsei evangélikus gimnázium vizsg...,NaN,NaN,vizsga,NaN,1646. szeptember 9. A tételeket szeptember 25-...,2147


A lőcsei vonatkozású alkalmi nyomtatványok-tábla "címzettek" és "szerzők" oszlopait átolvasva, a nevekhez hozzárendeljük a személyek-adattábla megfelelő elemét, a kettőből relációs táblát készítve:

In [12]:
def get_from_citizen_list(name, record):
    
    name=name.replace('-', '')
    name=name.replace('[', '')
    name=name.replace(']', '')
    name=name.strip()
    name=name.rstrip()
    
    
    index_name = citizens['név'].str.contains(name, na=False)
    index_alt_name = citizens['névváltozat'].str.contains(name, na=False)
    
    
    index_list = citizens[index_name]
    index_alt_list = citizens[index_alt_name]
    
    
    filtered_by_record = index_list['RMNy (alkalmi)'].astype(str).str.contains(str(record))
    filtered_by_record_alt = index_alt_list['RMNy (alkalmi)'].astype(str).str.contains(str(record))
    if not filtered_by_record.empty:
        return filtered_by_record.index[0]
    if not filtered_by_record_alt.empty:
        return filtered_by_record_alt.index[0]
    

    return math.nan


relations = []
counter = { }

for index, record in records.iterrows():
    recipient_names = record['a nyomtatvány címzettjei']
    sender_names = record['szerzők és a nyomtatványban említett kapcsolódó nevek']
    
    if isinstance(recipient_names, str) and isinstance(sender_names, str):
        recipients = recipient_names.split(';') 
        senders = sender_names.split(';')
        for recipient in recipients:
            target_id = get_from_citizen_list(recipient, index)
            
            for sender in senders:
                sender_id = get_from_citizen_list(sender, index)
                relation = {}
                relation['record_id'] = index
                relation['sender_name'] = sender
                if not math.isnan(sender_id):
                    name = str(citizens.iloc[sender_id]['név'])
                    relation['sender_id'] = int(sender_id)
                    relation['sender_name'] = name
                    if name not in counter:
                        counter[name] = {
                            'sender': 0,
                            'target': 0
                        }
                    counter[name]['sender'] += 1
                else:
                    relation['sender_id'] = -1
                
                relation['target_name'] = recipient
                if not math.isnan(target_id):
                    name = str(citizens.iloc[target_id]['név'])
                    relation['target_id'] = int(target_id)
                    relation['target_name'] = name
                    
                    if name not in counter:
                        counter[name] = {
                            'sender': 0,
                            'target': 0
                        }
                    counter[name]['target'] += 1
                    
                else:
                    relation['target_id'] = -1
                    
                
                relations.append(relation)

                

df = pandas.DataFrame.from_dict(counter).transpose()
df['sum'] = df['sender']+df['target']
df = df.sort_values(by=['sum'], ascending=False)
df = df.head().style.set_caption("A nyomtatványok között leggyakrabban előforduló nevek")
df

,sender,target,sum
"Seelmann, Christian (1626-1675/76)",28,92,120
"Zabeler, Hiob (1628–1664)",8,92,100
"Zauchtler, Anna Catharina (1635-1668)",0,73,73
"Frölich, Anna Euphrosina (?-?)",2,66,68
"Böhm, Christoph (1626-1660)",0,68,68


In [13]:
relations_df = pandas.DataFrame.from_records(relations)
relations_df.head()

,record_id,sender_name,sender_id,target_name,target_id
0,1076,"Holtzmann, Stephanus (1572-1619)",222,Thurzó Kristóf (?-1614),552
1,1230,"Musculus, Wolfgang (?-1638)",364,"Cramer, Caspar id. (?-1644)",77
2,1231,"Zabeler, Peter id. (1578-1645)",597,"Holtzmann, Stephanus (1572-1619)",222
3,1231,"Horlerus, Stanislaus (?-1645k.)",223,"Holtzmann, Stephanus (1572-1619)",222
4,1254,"Musculus, Wolfgang (?-1638)",364,"Cramer, Daniel (? -?)",80


Az alábbiakban a relációs táblából a networkx könyvtár segítségével relációs gráfot készítek.

In [14]:
G = nx.from_pandas_edgelist(relations_df, source='sender_name', target='target_name', edge_attr=True, create_using=nx.MultiDiGraph())

Az alábbiakban meghatározzuk az eseménytípusok színkódjait:<br>
 <span style="color: black"><strong> Halál - fekete</strong></span><br>
 <span style="color: purple"><strong>Esküvő - lila</strong></span><br>
 <span style="color: red"><strong>Beiktatás - piros</strong></span><br>
 <span style="color: blue"><strong>Általános üdvözlés - kék</strong></span><br>
 <span style="color: green"><strong>Vizsga - zöld</strong></span><br>
 <span style="color: gold"><strong>Program - sárga</strong></span><br>
 <span style="color: orange"><strong>Születésnap - narancssárga</strong></span><br>
 <span style="color: brown"><strong>Könyv - barna</strong></span><br>
 <span style="color: lightgrey"><strong>Alapértelmezett - szürke</strong></span><br>
 

In [15]:
edge_colors = {
    'halál': 'black',
    'esküvő': 'purple',
    'beiktatás': 'red',
    'általános üdvözlés': 'blue',
    'vizsga': 'green',
    'program': 'yellow',
    'születésnap': 'narancssárga',
    'könyv': 'brown',
    'default': 'grey'
}

def get_edge_color(type):
    if type in edge_colors.keys():
        return edge_colors[type]
    else:
        return edge_colors['default']


A megjelenített személyhez hozzáadjuk a rövid leírást, melyet a pontokra való kattintáskor láthatunk. Ezen kívül minden relációhoz hozzárendeljük a nyomtatvány számát és rövid leírását.

In [47]:
def setup_network(net):
    for edge in net.edges:
        to_node = net.get_node(edge['to'])
        from_node = net.get_node(edge['from'])
        to_node['title'] = "Nem található leírás."
        from_node['title'] = "Nem található leírás."
    
        record_id = edge['record_id']
    
        edge['title'] = str(record_id) + '\n' + textwrap.fill(records.loc[record_id]['a nyomtatvány rövid leírása']) 
        edge['color'] = get_edge_color(records.loc[record_id]['esemény'])
    
        if edge['sender_id'] != -1:
            string_sender = citizens.iloc[edge['sender_id']]['rövid leírás']
            if isinstance(string_sender, str):
                from_node['title'] = str(edge['from']) + '\n' + textwrap.fill(string_sender, 40)
        if edge['target_id'] != -1:
            string_target = citizens.iloc[edge['target_id']]['rövid leírás']
            if isinstance(string_target, str):
                to_node['title'] = str(edge['to']) + '\n' + textwrap.fill(string_target, 40)
    
        
        
net = network.Network(notebook=True, directed=True)
net.from_nx(G)
setup_network(net)
net.toggle_physics(True)
net.show('relationship.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [ ]:
A következőkben az adattáblák kapcsolatait felhasználva tetszőleges személyek karrierútját és kapcsolathálóját vázolhatjuk fel.

In [44]:
def get_relations_of_person(name):
    person_df = relations_df[relations_df.applymap(lambda x: name in str(x)).any(1)]
    df3=pandas.merge(person_df, records, left_on="record_id",right_on="RMNy")
    grouped_by_record_id = df3.groupby('record_id').groups


    chronology = []
    for record_id in grouped_by_record_id:
        new_item={}
        new_item['id'] = record_id
        new_item['kapcsolat'] = ''
        for id in grouped_by_record_id[record_id]:
            item = df3.iloc[id]
            new_item['év'] = item['év']
            new_item['esemény'] = item['esemény']
            new_item['a nyomtatvány rövid leírása'] = item['a nyomtatvány rövid leírása']
            if item['sender_name']==name:
                new_item['irány']='küldő'
                new_item['kapcsolat']+=item['target_name']+'; '
            elif item['target_name']==name:
                new_item['irány']='fogadó'
                new_item['kapcsolat']+=item['sender_name']+'; '   
        chronology.append(new_item)
    
    return pandas.DataFrame.from_records(chronology)
    
    
    
name = 'Seelmann, Christian (1626-1675/76)'
chronology_df = get_relations_of_person(name)

display(
    HTML(
        "<h3>"+ name + " lőcsei kapcsolatai: </h3><br><div style='height: 300px; overflow: auto; width: fit-content'>" +
        chronology_df[['év', 'irány', 'esemény', 'kapcsolat', 'id', 'a nyomtatvány rövid leírása',]].sort_values('év').to_html() +
        "</div>"
    )
)

,év,irány,esemény,kapcsolat,id,a nyomtatvány rövid leírása
0,1664,küldő,esküvő,"Zabeler, Hiob (1628–1664); Andreae, Maria (?-?);",3163,Lakodalmi versek Hiob Zabeler és Maria Andreae házasságkötésére 1664 május 5-én. A párt barátok és jóakarók köszöntötték. (2.)
1,1665,fogadó,beiktatás,"Augustini, Joannes (1610k.-1675); Brewer, Lorenz (1590-1664); Brewer, Samuel (1645-1699); Kalinka, Joachim (1601-1678); Sartorius, Andreas id. (1624-1679); Tarnóci, Martin (1620-1685);",3209,Trencsén megyei magas rangú egyházi személyek köszöntötték a Körmöcbányáról a lőcsei első lelkészi hivatalra meghívott Christian Seelmann evangélikus lelkészt
2,1665,küldő,esküvő,"Kemmel, Johann (1636-1685); Stürbitz, Regina (?-?);",3212,"A nyomtatvány Johann Kemmelnek, a lőcsei gimnázium rektorának, a teológia tanárának és Regina Stürbitznek, Andreas Stürbitz kassai tanácsos árvájának esküvőjére készült."
3,1665,fogadó,beiktatás,"Fabri, Daniel (?-?); Höher, Matthias (?-1684);",3220,A Körmöcbányáról a lőcsei első lelkészi hivatalra meghívott Christian Seelmann evangélikus lelkészt köszöntötték.
4,1665,fogadó,beiktatás,"Bussaeus, Bartholomeus (1625k.-1665u.); Kreuchel, Elias (?-?); Titius, Gottfried (?-?); Windisch, Johann (1605-1672);",3221,"Christian Seelmann evangélikus lelkészt és iskolafelügyelőt abból az alkalomból köszöntötték selmecbányai barátai, hogy Körmöcbányáról a lőcsei evangélikus egyház hívta meg első lelkészének."
5,1665,fogadó,beiktatás,"Greschner, Elias (?-?); Greschner, Matthias (?-?); Heinzel, Joannes (?-?); Láni, Daniel (1605-1665); Porsch, Matthias (?-1676); Schütz, Georg (?-?); Steller, Thomas (1640-1715); Stürtzer, Matthias (?-?);",3222,A Lőcsére elhívott Christian Seelmannt köszöntötték besztercebányai egyházi személyek.
6,1665,küldő,esküvő,"Kayser, Georgius Udalricus (?-?); Sock, Judith (?-?);",3223,"Christian Seelmann két költeménnyel köszöntötte Georgius Udalricus Kayser császári és királyi tanácsost, selmecbányai szenátort és bányatulajdonost, Hont vármegye ülnökét, valamint Judith Sockot egybekelésük alkalmából."
7,1665,fogadó,beiktatás,"Serpilius, August (1643-1699);",3224,"Christian Seelmann, volt körmöcbányai lelkész 1665. június 12-én elfoglalta lőcsei lelkipásztori állását. A címlevél után következő vers 12 számozott versszakból áll."
8,1665,fogadó,beiktatás,"Tieftrunck, Daniel (1629-1681);",3225,"Daniel Tieftrunck soproni rektor abból az alkalomból köszöntötte Christian\nSeelmannt, hogy az 1665. június 12-én elfoglalta lőcsei lelkészi állását."
9,1666,küldő,program,szervezet a lőcsei evangélikus gimnázium ünnepi programja 1666 január 30.;,3295,"A lőcsei evangélikus gimnázium 1666. január 30-i ünnepélyének programja, amelyen a tanulók szónoki tudományukról tettek bizonyságot."


In [43]:
person_df = relations_df[relations_df.applymap(lambda x: name in str(x)).any(1)]
christian_G = nx.from_pandas_edgelist(person_df, source='sender_name', target='target_name', edge_attr=True, create_using=nx.MultiDiGraph())
net2 = network.Network(notebook=True, directed=True)
net2.from_nx(christian_G)
setup_network(net2)
net2.toggle_physics(True)
net2.show('christian_seelmann.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [45]:
name = 'Alauda, Josephus (1610 k.-1664)'
chronology_df = get_relations_of_person(name)

display(
    HTML(
        "<h3>"+ name + " lőcsei kapcsolatai: </h3><br><div style='height: 300px; overflow: auto; width: fit-content'>" +
        chronology_df[['év', 'irány', 'esemény', 'kapcsolat', 'id', 'a nyomtatvány rövid leírása',]].sort_values('év').to_html() +
        "</div>"
    )
)

,év,irány,esemény,kapcsolat,id,a nyomtatvány rövid leírása
0,1635,küldő,esküvő,"Frölich, David (1595-1648); Bertram, Judith (?-?);",1622,Lakodalmi köszöntő versek David Frölich és Judith Bertram menyegzőjére több szerzőtől. 1635.
1,1635,küldő,esküvő,"Alauda, Bartholomeus (?-?); Lapscher, Susanna (?-?);",1625,"Bartholomeus Alauda, a lőcsei szlovák evangélikusok lelkipásztorának és Andreas Hertel, kisszebeni tanácsos és jegyző Susanna Lapscher nevű mostohalányának menyegzője alkalmával készült."
2,1637,fogadó,általános üdvözlés,Ismeretlen szerző I. (?-?);,1681,"A művelt szerző köszvény miatt pihenésre kényszerítve írt verseket lőcsei polgárokhoz, de magát nem nevezte meg."
3,1638,küldő,halál,"Lang, Anton (1617-1638);",1731,A verseket Anton Láng halála alkalmából írták a lőcsei iskola\ntanárai.
4,1640,küldő,esküvő,"Windisch, Johann (1605-1672); Zabeler, Susanna (?-?);",1837,"Az esküvőt 1640. november 4-én tartották, a köszöntő versek írói a lőcsei iskola tanárai és tanulói."
5,1642,küldő,esküvő,"Zabeler, Peter id. (1578-1645); Goltz, Marta (?-?);",1936,A költemények Peter Zabeler és Marta Goltz 1642. november 2-án tartott esküvőjére\nkészültek.
6,1647,küldő,beiktatás,"Schlegel, Christoph (1613-1678);",2177,1647-ben írt üdvözlő versek Christoph Schlegel beiktatása alkalmából. (Első pap)
7,1648,küldő,esküvő,"Schlegel, Christoph (1613-1678); Gloger, Rosina (?-?);",2232,Christoph Schlegel és Rosina Gloger 1648 október 25-én tartott eskövőjére készült köszöntő versek.
8,1651,küldő,esküvő,"Zabeler, Hiob (1628–1664); Milleter, Margareta (?-?);",2374,"A lakodalmi köszöntő versek Hiob Zabeler lőcsei rektor és Margareta Milleter (Georgius Milleter, szepesi gróf lánya) 1651-ben tartott esküvőjére készültek."
9,1653,küldő,esküvő,Horváth Márk (?-?); Máriássy Zsófia (?-?);,2469,A kiadvány Horváth Márk és Máriássy Zsófia házasságkötésére készült 1653. február 9-én.


In [46]:
person_df = relations_df[relations_df.applymap(lambda x: name in str(x)).any(1)]
josephus_G = nx.from_pandas_edgelist(person_df, source='sender_name', target='target_name', edge_attr=True, create_using=nx.MultiDiGraph())
net2 = network.Network(notebook=True, directed=True)
net2.from_nx(josephus_G)
setup_network(net2)
net2.toggle_physics(True)
net2.show('alauda_josephus.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
